Exécuter ce code pour importer toutes les données scrapées ci-dessous

In [22]:
import pickle,zlib,pandas as pd

player_stats_fbref=pickle.loads(zlib.decompress(open('fbref_data/compressed_player_stats_fbref_1.bin','rb').read()))
player_stats_fbref.update(pickle.loads(zlib.decompress(open('fbref_data/compressed_player_stats_fbref_2.bin','rb').read())))
player_stats_fbref=pd.DataFrame(player_stats_fbref).transpose()
player_stats_fbref=player_stats_fbref.sort_index(axis=1,ascending=False)

teams_fbref=pickle.load(open('fbref_data/teams_fbref.pkl','rb'))

glossary=pd.read_csv('fbref_data/Glossaire_FBREF.csv')


player_stats_tm=pd.read_csv('tm_data/player_stats_tm.csv')
player_stats_tm.index=player_stats_tm.loc[:,'Unnamed: 0']
player_stats_tm=player_stats_tm.drop('Unnamed: 0',axis=1)

transfer_history=pickle.load(open('tm_data/transfer_history.pkl','rb'))

# Données FBREF

## Équipes 

(!= Selenium pour être sur que le site reconnaisse ex: Real Betis != Real Betis Balompié)

Module pour Scrapper, (on peut aussi utiliser le truc std de python ie html_parser)

In [ ]:
!pip install lxml

In [241]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

def get_teams_fbref(champ,season):
    
    #Code HTML de la page fbref de la saison season pour le championnat champ
    url=f'https://fbref.com/en/comps/{champ}/{season}-{season+1}/{season}-{season+1}-any-Stats'
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    
    # Trouver le classement et toutes les lignes du tableau
    table=soup.find_all('table')[0]
    rows=table.find_all('tr')[1:]
    res={}
    
    #Faire un dicto avec toutes les équipes et leur id (utile pour après)
    for row in rows:
        link=row.find('a').get('href')
        id=link.split('/')[3]
        name=row.find('a').text
        res[name]=id
    return res

    

Trouver toutes les équipes pour tous les championnats sur les 10 dernières années

In [ ]:
championships={'GB1':9,'IT1':11,'ES1':12,'FR1':13,'L1':20}
years=[2024-i for i in range(10)]



teams_fbref={}


for c in championships.keys():
    teams_fbref[c]={}
    for year in years:
        print(c,year)
        teams_fbref[c][year]=get_teams_fbref(championships[c],year)
        time.sleep(4)


In [2]:
teams_fbref['GB1']['2024']

NameError: name 'teams_fbref' is not defined

In [239]:
get_teams_fbref(championships['L1'],2015)

{'1 Bayern Munich3428428017+63882.5975,000Robert Lewandowski - 30Manuel Neuer→ Champions League via league finish': '054efa67',
 '2 Dortmund3424648234+48782.2981,178Pierre-Emerick Aubameyang - 25Roman Bürki→ Champions League via league finish': 'add600ae',
 '3 Leverkusen34186105640+16601.7629,085Javier Hernández - 17Bernd Leno→ Champions League via league finish': 'c7a9f859',
 '4 Gladbach34174136750+17551.6251,715Raffael - 13Yann Sommer→ Champions League via league finish': '32f3ee20',
 '5 Schalke 0434157125149+2521.5361,386Klaas-Jan Huntelaar - 12Ralf Fährmann→ Europa League via league finish': 'c539e393',
 '6 Mainz 0534148124642+4501.4731,053Yunus Mallı - 11Loris Karius→ Europa League via league finish 1': 'a224b06a',
 '7 Hertha BSC341481242420501.4749,704Salomon Kalou - 14Rune Jarstein→ Europa League via league finish': '2818f8bc',
 '8 Wolfsburg34129134749-2451.3228,945André Schürrle - 9Diego Benaglio': '4eaa11d7',
 '9 Köln341013113842-4431.2648,676Anthony Modeste - 15Timo Horn': 'b

Sauvegarde du dictionnaire des équipes

In [425]:
import pickle
with open('teams_fbref.pkl','wb') as file: 
    pickle.dump(teams_fbref,file)

### Glossaire pour expliquer les acronymes des différentes stats

Obligé d'utiliser Selenium car présence d'un boutton

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time



#Sélecteurs de tous les bouttons

button_paths=['#stats_shooting_dom_lg_sh','#stats_passing_dom_lg_sh', \
              '#stats_passing_types_dom_lg_sh','#stats_gca_dom_lg_sh', \
                '#stats_playing_time_dom_lg_sh','#stats_possession_dom_lg_sh', '#stats_misc_dom_lg_sh']

button_paths=[button+f' > div > ul > li:nth-child({1 if button_paths.index(button)!=0 else 2}) > button' for button in button_paths]

#À titre d'exemple (on aurait pu prendre un autre joueur)

url=f'https://fbref.com/en/players/e06683ca/Virgil-van-Dijk'



driver=webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(url)

glossaries=[]


for button in button_paths:
  
  # Scroller jusqu'au boutton glossaire, le code ajustera jusqu'à trouver le boutton par lui même
    while(True):
      try:
        driver.find_element(By.CSS_SELECTOR,button).click()
        break
      except:
        pass
    glossaries.append(([x.text for x in driver.find_elements(By.XPATH,'//*[@id="sr_glossary"]')]))
    

In [ ]:
glossary={}
for i in range(len(glossaries)):
    stats=[s.split('\n')[-1] for s in glossaries[i][0].split('--')][:-1 ]
    explanations=[s.split('\n')[0] for s in glossaries[i][0].split('--')][1:]
    stats=list(map(lambda s:s.strip(),stats))
    explanations=list(map(lambda s:s.split(':')[0].strip(),explanations))
    glossary.update(zip(stats,explanations))
glossary
    

Sauvegarde du glossaire

In [40]:
import pandas as pd 
glossary=pd.Series(glossary)
glossary.to_csv('Glossaire_FBREF.csv')

In [389]:
glossary.head(10)

Season                            Season
Age                  Age at season start
Country    Flag and name of the country.
Comp                         Competition
LgRank       Squad finish in competition
90s                           90s Played
Gls                                Goals
Sh                            GCA (Shot)
SoT                      Shots on Target
SoT%                   Shots on Target %
dtype: object

Stats des joueurs

In [345]:
def get_player_stats_fbref(player_id):
    url=f'https://fbref.com/en/players/{player_id}/'
    html = requests.get(url).text
    time.sleep(4)
    soup = BeautifulSoup(html, 'lxml')
    res={}
    #Données sur taille poids pied fort âge et pays
    
    metadata=soup.find('div',id='meta')
    try:
        physical=list(map(lambda x:x.text,metadata.find_all('span')[1:3]))
        physical[0],physical[1]=float(physical[0].split('cm')[0]),float(physical[1].split('kg')[0])
    except:
        physical=['','']
    try:
        foot=metadata.find('strong',string='Footed:').next_sibling.strip()
    except:
        foot=''
    try:
        age=2025-int(metadata.find('span',id='necro-birth').get('data-birth').split('-')[0])
    except:
        age=''
    try:
        country=metadata.find('a').text
    except:
        country=''
    meta=physical+[foot,age,country]
    
    
    
    
    attributes=['height','weight','preferred foot','age','country']
    
    res['meta']=dict(zip(attributes,meta))
    
    
    
    
    
    std_rows=soup.find('table',id="stats_standard_dom_lg")
    
    try:
        std_rows=std_rows.select('tr[id="stats"]')
    except:
        return res
    
    
    # Données sur les antécédents en club, pays, championnats
    teams,countries,champs=[],[],[]
    for row in std_rows:
        teams.append(row.select_one(('td[data-stat="team"]')).text)
        countries.append((row.select_one(('td[data-stat="country"]'))).text.split(' ')[-1] )
        champs.append(row.select_one(('td[data-stat="comp_level"]')).text.split('1. ')[-1])
        
    # Extraction des stats sur tous les aspects du jeu
    
    
    #On différencie les Gardiens
    
    try:
        if soup.find(string='Position:').next_element[1]=='G':
            table_ids=["keeper","keeper","passing","passing_types","possession","playing_time","misc"]
        else:
            table_ids=["shooting","passing","passing_types","gca","defense","possession","playing_time","misc"]
    except:
        table_ids=["shooting","passing","passing_types","gca","defense","possession","playing_time","misc"]
    table_ids=list(map(lambda s:'stats_'+s+'_dom_lg',table_ids))
    
    
    
    for id in table_ids:
        
        
        #Certains joueurs n'ont pas des tableaux pour tous les aspects du jeu
        try:
            table=soup.find('table',id=id)
            header=table.find_all('tr')[1]
            cols=[x.get('data-stat') for x in header.find_all('th')][1:]
            for row in table.find_all('tr',id="stats"):
                year=row.select_one('th[data-stat="year_id"]').text
                if year not in res.keys():
                    res[year]={}
                for col in cols[:-1]:
                    res[year][col]=row.select_one(f'td[data-stat="{col}"]').text
        except:
            pass
        
    # Transformation en flottants pour les nombres
    for y in res.keys():
        
        for c in res[y].keys():
            try:
                res[y][c]=float(res[y][c])
            except:
                pass
    
    return res
        
    
def get_team_stats_fbref(team_id,season,champ,player_dict={},display_flag=False):
    url=f'https://fbref.com/en/squads/{team_id}/{season}-{season+1}/any'
    html = requests.get(url).text
    time.sleep(4)
    soup = BeautifulSoup(html, 'lxml')
    
    
    table=soup.find('table',id=f"stats_standard_{championships[champ]}").find('tbody')
    for row in table.find_all('tr'):
        name=row.find('a').text
        if name not in player_dict.keys():
            if display_flag:
                print(name)
            player_dict[name]=get_player_stats_fbref(row.find('a').get('href').split('/')[3])
    
    
    
    
    
    
    
    

In [ ]:
player_stats_fbref={}

#Seen pour relancer sans devoir reparcourir toutes les équipes si erreur
seen=set()
for country in teams_fbref.keys():
    for year in teams_fbref[country].keys():
        for team in teams_fbref[country][year].keys():
            if (team,year) not in seen:
                print(team,year)
                get_team_stats_fbref(teams_fbref[country][year][team],year,country,player_stats_fbref)
                seen.add((team,year))
                
                

Compression pour pouvoir mettre les données sur Github

In [420]:
import zlib
n=len(player_stats_fbref)

#Séparation en 2 pour respecter la limite des 25 Mb de Github
player_stats_fbref_1=dict(list(player_stats_fbref.items())[int(n/2):])
player_stats_fbref_2=dict(list(player_stats_fbref.items())[:int(n/2)])

compressed_player_stats_fbref_1=zlib.compress(pickle.dumps(player_stats_fbref_1))
compressed_player_stats_fbref_2=zlib.compress(pickle.dumps(player_stats_fbref_2))


with open('compressed_player_stats_fbref_1.bin', 'wb') as f:
    f.write(compressed_player_stats_fbref_1)
with open('compressed_player_stats_fbref_2.bin', 'wb') as f:
    f.write(compressed_player_stats_fbref_2)



In [3]:


with open('compressed_player_stats_fbref_1.bin', 'rb') as f:
    compressed_player_stats_fbref_1=f.read()
with open('compressed_player_stats_fbref_2.bin', 'rb') as f:
    compressed_player_stats_fbref_2=f.read()


#Pour être sur que la compression s'est bien passée
player_stats_fbref=pickle.loads(zlib.decompress(compressed_player_stats_fbref_1))
player_stats_fbref.update(pickle.loads(zlib.decompress(compressed_player_stats_fbref_2)))

In [27]:
# Sauvegarde au format csv

player_stats_fbref=pd.DataFrame(player_stats_fbref).transpose()
player_stats_fbref=player_stats_fbref.sort_index(axis=1,ascending=True)

In [28]:
player_stats_fbref.head()

,meta,2024-2025,2024,2023-2024,2023,2022-2023,2022,2021-2022,2021,2020-2021,...,1999-2000,1999,1998-1999,1998,1997-1998,1996-1997,1995-1996,1994-1995,1993-1994,1992-1993
,"{'height': '', 'weight': '', 'preferred foot':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Ciammaglichella,"{'height': '', 'weight': '', 'preferred foot':...","{'age': 19.0, 'team': 'Torino', 'country': 'it...",NaN,"{'age': 18.0, 'team': 'Torino', 'country': 'it...",NaN,"{'age': 17.0, 'team': 'Torino', 'country': 'it...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Connolly,"{'height': 180.0, 'weight': 77.0, 'preferred f...","{'age': 24.0, 'team': 'Sunderland', 'country':...",NaN,"{'age': 23.0, 'team': 'Hull City', 'country': ...",NaN,"{'age': 22.0, 'team': 'Hull City', 'country': ...",NaN,"{'age': 21.0, 'team': 'Middlesbrough', 'countr...",NaN,"{'age': 20.0, 'team': 'Brighton', 'country': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Cresswell,"{'height': 170.0, 'weight': 68.0, 'preferred f...","{'age': 34.0, 'team': 'West Ham', 'country': '...",NaN,"{'age': 33.0, 'team': 'West Ham', 'country': '...",NaN,"{'age': 32.0, 'team': 'West Ham', 'country': '...",NaN,"{'age': 31.0, 'team': 'West Ham', 'country': '...",NaN,"{'age': 30.0, 'team': 'West Ham', 'country': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aaron Herzog,"{'height': '', 'weight': '', 'preferred foot':...",NaN,NaN,"{'age': 25.0, 'team': 'Lübeck', 'country': 'de...",NaN,"{'age': 24.0, 'team': 'Hallescher', 'country':...",NaN,"{'age': 23.0, 'team': 'Hallescher', 'country':...",NaN,"{'age': 22.0, 'team': 'Hansa Rostock', 'countr...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
!mkdir fbref_data
!mv compressed_player_stats_fbref_1.bin compressed_player_stats_fbref_2.bin Glossaire FBREF Glossaire_FBREF.csv teams_fbref.pkl fbref_data


# Données Transfermarkt

## Classements des 10 dernières années dans les 5 grands championnats

## Données sur les joueurs

(On récupère évolution de la valeur marchande + postes)

In [ ]:
#Si nécessaire
!pip3 install rapidfuzz

In [72]:
player_stats_fbref.loc['Mohamed Salah','2024-2025']['team']

'Liverpool'

In [96]:
na_player_stats_fbref=player_stats_fbref.isna()

In [244]:
# Utile pour trouver gommer les différences avec l'équipe tm correspondante
teams_fbref_list=[]
for k,v in teams_fbref.items():
    for kk,vv in v.items():
        teams_fbref_list+=(list(vv.keys()))

In [ ]:
teams_fbref

In [312]:
import time
from rapidfuzz.distance import Levenshtein
#Header pout ne pas se faire bloquer l'accès par le site
heads = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'}
#Extraction individuelle
def get_player_stats_tm(player_id):
    url=f'https://www.transfermarkt.com/konstantinos-tsimikas/profil/spieler/{player_id}'
    
    
    html = requests.get(url,headers=heads).text
    time.sleep(1)
    soup = BeautifulSoup(html, 'lxml')
    
    # Extraction des positions
    position_box=soup.find('div',class_='detail-position__box')
    try:
        positions=position_box.find_all('dd',class_='detail-position__position')
        positions=[position.text for position in positions]
    except:
        positions=[]
    
    
    
    # Extraction de l'histoirique de la valeur marchande (accessible à travers un endpoint)
    
    
    value_graph=requests.get(f'https://transfermarkt.com/ceapi/marketValueDevelopment/graph/{player_id}',headers=heads)
    time.sleep(1)
    value_graph=value_graph.json()
    value_graph_values=[d['mw'] for d in value_graph['list']]
    
    value_graph_values=[float(v[1:-1])*(1000 if v[-1]=='m' else 1) if v!='-' else 0. for v in value_graph_values ]
    value_graph_years=[d['datum_mw'].split(' ')[-1] for d in value_graph['list']]
    value_graph={value_graph_years[i]:value_graph_values[i] for i in range(len(value_graph_years))}
    return {'positions':positions,'market_values':value_graph}
    

#Extraction d'une équipe

def get_team_stats_tm(url,season,team,player_dict={},display_flag=False):
    html=requests.get(url,headers=heads).text
    time.sleep(1)
    soup=BeautifulSoup(html,'lxml')
    
    
    
    
    table=soup.find('div',id='yw1')
    table=table.find('tbody')
    rows=table.select('td.hauptlink:not(.rechts)')
    player_list=[]
    
    # Pour trouver l'équipe du joueur correspondante dans la bd de fbref
    team_tm=min([(Levenshtein.distance(team,team_fbref),team_fbref) for team_fbref in teams_fbref_list])[1]
    
    
    #Extraction de la bd des joueurs de l'équipe team_tm lors de season dans fbref
    
    # Pas forcément nécessaire mais pour considérer les cas où la saison est au format année ( en Amérique du Sud nottament) ou année - année + 1 (cas usuel en Europe)
    team_fbref_1=[player for player in player_stats_fbref.index if not(na_player_stats_fbref.loc[player,str(season)]) and player_stats_fbref.loc[player,str(season)]['team']==team_tm]
    team_fbref_2=[player for player in player_stats_fbref.index if not(na_player_stats_fbref.loc[player,str(season)+'-'+str(season+1)]) and player_stats_fbref.loc[player,str(season)+'-'+str(season+1)]['team']==team_tm]
    if len(team_fbref_2)>=len(team_fbref_1):
        season=str(season)+'-'+str(season+1)
        team_fbref=team_fbref_2
    else:
        season=str(season) 
        team_fbref=team_fbref_1
        
    # Pour chaque joueur on trouve le joueur correspondant dans la bd fbref
    for row in rows:
        name_tm=row.find('a').text.strip()
        score,name=min([(Levenshtein.distance(player,name_tm),player) for player in team_fbref])
        player_list.append((score,name,row.find('a').get('href')))
    
    
    # On priorise les joueurs qui ont probablement un match dans la bd fbref
    # (si la distance de Levenshtein est grande pour tous les joueurs à priori ce joueur n'est pas dans la bd fbref)
    player_list=sorted(player_list)
    player_list=player_list[:len(team_fbref)]
    for score,name,link in player_list:
        
        #Pour éviter les redondances
        if name not in player_dict.keys():
            if display_flag:
                print(name)
            player_dict[name]=get_player_stats_tm(link.split('/')[-1])
    
    
        
    


#Extraction d'un championnat entier
def get_champ_stats_tm(season,champ,player_dict={},display_flag=False):
    url=f'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/{champ}/plus/?saison_id={season}'
    html=requests.get(url,headers=heads).text
    time.sleep(1)
    soup=BeautifulSoup(html,'lxml')
    table=soup.find('div',id='yw1')
    
    table=table.find('tbody')
    rows=table.find_all('tr')
    for row in rows:
        team_url=row.find('a').get('href')
        team=row.find('a').get('title')
        if display_flag:
            print(team)
        get_team_stats_tm('https://www.transfermarkt.com'+team_url,season,team,player_dict,False)
            

        
        
        
        
        
        
        
        


In [ ]:
#player_stats_tm={}
#Seen pour relancer sans devoir reparcourir toutes les équipes si erreur
#seen=set()
for country in championships.keys():
    for year in years:
            if (country,year) not in seen:
                print(country,year)
                #Enlever display flag si besoin
                get_champ_stats_tm(year,country,player_stats_tm,display_flag=True)
                seen.add((country,year))
                
                

Sauvegarde des données

In [320]:
pickle.dump(player_stats_tm,open('player_stats_tm.pkl','wb'))

In [323]:
player_stats_tm_dict=player_stats_tm
player_stats_tm=pd.DataFrame(player_stats_tm).transpose()
player_stats_tm.to_csv('player_stats_tm.csv',index=False)

In [43]:
for player in player_stats_tm_dict.keys():
    player_stats_fbref.loc[player,'meta'].update({'positions':player_stats_tm_dict[player]['positions'],'market_values':player_stats_tm_dict[player]['market_values']})
    
player_stats_tm.to_csv('player_stats_tm.csv',index=False)

## Extraction des historiques de transfert pour chaque équipe

In [426]:
def get_team_transfers(url,team,season,champ,transfer_dict={}):
    html=requests.get(url,headers=heads).text
    time.sleep(1)
    soup=BeautifulSoup(html,'lxml')
    
    
    
    arrivals_tab=soup.find('div',id='yw1').find('tbody').find_all('tr',{'class':['even','odd']})
    departures_tab=soup.find('div',id='yw2').find('tbody').find_all('tr',{'class':['even','odd']})
    arrivals,departures=[],[]
    
    #Utiles pour calculer le déficit/budget d'un club
    spent,sell=0,0
    
    
    for type in [arrivals_tab,departures_tab]:
        for transfer in type:
            links=transfer.find_all('a')
            name,club,fee=links[0].text,links[1].get('title'),links[-1].text
            #Différencier transfert monétaire et autres (prêts, transferts gratuits...)
            if fee[-1] in ['k','m']:
                fee=float(fee[fee.index('€')+1:-1])*(1000 if fee[-1]=='m' else 1)
                if type==arrivals_tab:
                    spent+=fee
                else:
                    sell+=fee
            (arrivals if type==arrivals_tab else departures).append([name,club,fee])
        
    for type in [arrivals,departures]:
        for name,club,fee in type:
            
            #On cherche les équivalents dans la bd fbref 
            
            #Pour les retraités, ceux qui arrêtent le foot
            if club== None:
                club_fbref=None
            else:
                club_fbref=min([(Levenshtein.distance(club,club_f),club_f) for club_f in teams_fbref_list])[1]
            
            name_fbref=min([(Levenshtein.distance(name,name_f),name_f) for name_f in player_stats_fbref.index])[1]
            type.remove([name,club,fee])
            type.append([name_fbref,club_fbref,fee])
    transfer_dict[champ][season][team]={'arrivals':arrivals,'departures':departures,'spent':spent,'sell':sell}

    
def get_champ_transfers(champ,season,transfer_dict={},display_flag=False):
    # On applique simplement la fonction précédente sur un championnat entier
    url=f'https://www.transfermarkt.com/any/startseite/wettbewerb/{champ}/plus/?saison_id={season}'
    html=requests.get(url,headers=heads).text
    time.sleep(1)
    soup=BeautifulSoup(html,'lxml')
    rows=soup.find('div',id='yw1').find('tbody').find_all('tr',{'class':["odd","even"]})
    for row in rows:
        team_id=row.find('a').get('href').split('/')[4]
        team_url=f'https://www.transfermarkt.com/any/transfers/verein/{team_id}/saison_id/{season}'
        team_name=row.find('a').get('title')
        if display_flag:
            print(team_name)
        get_team_transfers(team_url,team_name,season,champ,transfer_dict)
        
        
    
    
        
    

In [ ]:
transfer_history={}
seen=set()
for c in teams_fbref.keys():
    transfer_history[c]=transfer_history.get(c,{})
    for y in teams_fbref[c].keys():
        if (c,y) not in seen:
            print(c,y)
            transfer_history[c][y]={}
            get_champ_transfers(c,y,transfer_history)
            seen.add((c,y))
        
            
            
            

In [431]:
pickle.dump(transfer_history,open('transfer_history.pkl','wb'))

In [433]:
#!mkdir tm_data
!mv player_stats_tm.csv player_stats_tm.pkl transfer_history.pkl tm_data

# Modèle

## Input:Vectorisation des données des joueurs

### 